## Compare results from TSS to RNA-seq results

In [1]:
import os
f_save = "Results/Figures/Table1/RNA_Accession.txt"
if not os.path.exists(os.path.dirname(f_save)):
    os.mkdir(os.path.dirname(f_save))

In [2]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
anno_gff=doc['annotation']


tss_annotation = doc['tss_annotation']


import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 13)


### 1.

In [3]:
cho_rna_file = doc["cho_rna_file"]
cho_rna = pd.read_csv(cho_rna_file,sep="\t",index_col=0)
cho_rna = cho_rna.iloc[:,1:]
#cho_rna = np.log10(cho_rna+1)
cho_rna

,1,2,3,4,5,6,7,8,9,10,...,SRR2922605,SRR2922606,SRR3401745,SRR3401746,SRR3401747,SRR3401748,SRR3401749,SRR3401750,SRR3401751,SRR3401752
gid,,,,,,,,,,,,,,,,,,,,,
gene_10,15.090907,14.502347,17.472863,18.938263,16.794809,16.433305,15.446144,15.924950,13.390410,16.717359,...,2.818250,0.467242,9.186139,11.062778,9.725346,9.121378,7.273079,8.870101,10.162889,9.457826
gene_100,0.245168,0.286629,0.118151,0.292011,0.300439,0.176940,0.461869,0.548782,0.459613,0.582428,...,0.790368,0.082365,0.516127,0.714297,5.273478,13.265540,13.696956,21.196798,5.535234,17.286073
gene_10000_1,1.682718,1.522292,2.348422,1.786015,2.357684,1.998047,1.162090,0.373551,0.610389,0.615104,...,1.312794,0.167734,0.438990,0.126566,2.739962,1.797872,1.782853,1.061421,3.083283,1.745710
gene_10000_10,0.000000,0.000000,0.617603,0.779067,0.201739,0.000000,2.830664,0.000000,5.414581,2.351944,...,0.000000,0.000000,0.835100,2.498463,13.903834,4.886355,11.659910,8.752601,16.605154,14.407911
gene_10000_11,2.468370,2.586280,1.762731,2.337148,2.911244,3.379113,4.142082,3.166982,4.172392,6.558429,...,0.375830,0.004867,4.586689,2.230479,5.982803,2.710943,3.307470,3.586193,6.052315,6.061694
gene_10000_12,46.615873,45.918075,50.876039,58.640512,54.808573,53.177149,27.009581,29.059582,30.391605,31.605897,...,12.390022,1.335611,34.983115,29.271515,20.956637,15.055169,30.891023,29.715315,26.128494,14.819726
gene_10000_2,0.585002,0.518263,0.643621,0.749890,0.860604,0.857540,0.794382,1.316685,1.354184,1.316755,...,0.372291,0.057128,0.934583,0.563892,1.532557,0.694653,0.975335,1.059641,1.485217,1.320222
gene_10000_3,0.178900,0.070792,0.208639,0.170089,0.592712,0.186534,0.257439,0.165511,0.201603,0.189698,...,0.256481,0.066349,0.000000,0.187935,0.376137,0.301043,0.207470,0.000000,0.191237,0.106907
gene_10000_4,0.097433,0.136322,0.112180,0.067788,0.163007,0.038916,0.104102,0.301698,0.256680,0.000000,...,0.120444,0.005964,0.047602,0.153479,0.058165,0.153117,0.278629,0.159585,0.097551,0.117723


# Save the SRA values 

In [4]:
cho_rna

,1,2,3,4,5,6,7,8,9,10,...,SRR2922605,SRR2922606,SRR3401745,SRR3401746,SRR3401747,SRR3401748,SRR3401749,SRR3401750,SRR3401751,SRR3401752
gid,,,,,,,,,,,,,,,,,,,,,
gene_10,15.090907,14.502347,17.472863,18.938263,16.794809,16.433305,15.446144,15.924950,13.390410,16.717359,...,2.818250,0.467242,9.186139,11.062778,9.725346,9.121378,7.273079,8.870101,10.162889,9.457826
gene_100,0.245168,0.286629,0.118151,0.292011,0.300439,0.176940,0.461869,0.548782,0.459613,0.582428,...,0.790368,0.082365,0.516127,0.714297,5.273478,13.265540,13.696956,21.196798,5.535234,17.286073
gene_10000_1,1.682718,1.522292,2.348422,1.786015,2.357684,1.998047,1.162090,0.373551,0.610389,0.615104,...,1.312794,0.167734,0.438990,0.126566,2.739962,1.797872,1.782853,1.061421,3.083283,1.745710
gene_10000_10,0.000000,0.000000,0.617603,0.779067,0.201739,0.000000,2.830664,0.000000,5.414581,2.351944,...,0.000000,0.000000,0.835100,2.498463,13.903834,4.886355,11.659910,8.752601,16.605154,14.407911
gene_10000_11,2.468370,2.586280,1.762731,2.337148,2.911244,3.379113,4.142082,3.166982,4.172392,6.558429,...,0.375830,0.004867,4.586689,2.230479,5.982803,2.710943,3.307470,3.586193,6.052315,6.061694
gene_10000_12,46.615873,45.918075,50.876039,58.640512,54.808573,53.177149,27.009581,29.059582,30.391605,31.605897,...,12.390022,1.335611,34.983115,29.271515,20.956637,15.055169,30.891023,29.715315,26.128494,14.819726
gene_10000_2,0.585002,0.518263,0.643621,0.749890,0.860604,0.857540,0.794382,1.316685,1.354184,1.316755,...,0.372291,0.057128,0.934583,0.563892,1.532557,0.694653,0.975335,1.059641,1.485217,1.320222
gene_10000_3,0.178900,0.070792,0.208639,0.170089,0.592712,0.186534,0.257439,0.165511,0.201603,0.189698,...,0.256481,0.066349,0.000000,0.187935,0.376137,0.301043,0.207470,0.000000,0.191237,0.106907
gene_10000_4,0.097433,0.136322,0.112180,0.067788,0.163007,0.038916,0.104102,0.301698,0.256680,0.000000,...,0.120444,0.005964,0.047602,0.153479,0.058165,0.153117,0.278629,0.159585,0.097551,0.117723


## Take only Sample names that are not integers

In [5]:
labelled_cho_data = []
for i in cho_rna.columns.values:
    try:
        int(i)
    except ValueError:
         labelled_cho_data.append(i)

labelled_cho_data

with open(f_save,"w") as f:
    f.write("\n".join(labelled_cho_data))

In [6]:
labelled_cho_data

['ERR359637',
 'ERR359638',
 'ERR366009',
 'ERR366010',
 'SRR035274',
 'SRR035275',
 'SRR035276',
 'SRR035277',
 'SRR035278',
 'SRR035279',
 'SRR035280',
 'SRR035281',
 'SRR035282',
 'SRR035283',
 'SRR035284',
 'SRR035285',
 'SRR950107',
 'SRR950108',
 'SRR950109',
 'SRR1516214',
 'SRR1516215',
 'SRR1516216',
 'SRR1516217',
 'SRR2922597',
 'SRR2922598',
 'SRR2922599',
 'SRR2922600',
 'SRR2922601',
 'SRR2922602',
 'SRR2922603',
 'SRR2922604',
 'SRR2922605',
 'SRR2922606',
 'SRR3401745',
 'SRR3401746',
 'SRR3401747',
 'SRR3401748',
 'SRR3401749',
 'SRR3401750',
 'SRR3401751',
 'SRR3401752']